In [7]:
import tensorflow as tf
import numpy as np
import tensorflow.contrib.slim as slim
import imageio
import facenet

In [14]:
with tf.Graph().as_default():
        with tf.Session() as sess:
            
            facenet.load_model('/home/ovuser/FaceRecognition/models/20170512-110547')
            all_tensors = tf.contrib.graph_editor.get_tensors(tf.get_default_graph())
            all_tensor_names = [tensor.name for tensor in all_tensors]
        
            conv1_tensor_index = all_tensor_names.index('InceptionResnetV1/Conv2d_1a_3x3/weights:0')
            conv1_weights = all_tensors[conv1_tensor_index].eval()
            conv1_bn_mm_tensor_index = all_tensor_names.index('InceptionResnetV1/Conv2d_1a_3x3/BatchNorm/moving_mean:0')
            conv1_bn_mm = all_tensors[conv1_tensor_index].eval()
            conv1_bn_mv_tensor_index = all_tensor_names.index('InceptionResnetV1/Conv2d_1a_3x3/BatchNorm/moving_variance:0')
            conv1_bn_mv = all_tensors[conv1_tensor_index].eval()
            conv1_bn_beta_tensor_index = all_tensor_names.index('InceptionResnetV1/Conv2d_1a_3x3/BatchNorm/beta:0')
            conv1_bn_beta = all_tensors[conv1_tensor_index].eval()
            
            images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
    
            inputs = tf.placeholder(dtype = tf.float32, shape = (None, 160, 160, 3))
            with tf.variable_scope('Facenet', 'Facenet', [inputs], reuse=None):
                with slim.arg_scope([slim.batch_norm, slim.dropout],
                            is_training=False):
                    with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d],
                                stride=1, padding='SAME'):
      
                        # 149 x 149 x 32
                        net = slim.conv2d(inputs, 32, 3, stride=2, padding='VALID',
                                  scope='Conv2d_1a_3x3', normalizer_fn = slim.batch_norm)
                        
            init = tf.global_variables_initializer()
            sess.run(init)

            wts_tensor = tf.get_default_graph().get_tensor_by_name("Facenet/Conv2d_1a_3x3/weights:0")
            print (tf.shape(wts_tensor))
            tf.assign(wts_tensor, conv1_weights)
            mm_tensor = tf.get_default_graph().get_tensor_by_name("Facenet/Conv2d_1a_3x3/BatchNorm/moving_mean:0")
            tf.assign(mm_tensor, conv1_bn_mm)
            mv_tensor = tf.get_default_graph().get_tensor_by_name("Facenet/Conv2d_1a_3x3/BatchNorm/moving_variance:0")
            tf.assign(mv_tensor, conv1_bn_mv)
            beta_tensor = tf.get_default_graph().get_tensor_by_name("Facenet/Conv2d_1a_3x3/BatchNorm/beta:0")
            tf.assign(beta_tensor, conv1_bn_beta)
            output_tensor = tf.get_default_graph().get_tensor_by_name("InceptionResnetV1/Conv2d_1a_3x3/Relu:0")
            
            im = imageio.imread('/home/ovuser/Pictures/kduncan_48.png')
            images = [im]
            feed_dict = {inputs: images}
            emb_list = sess.run(net, feed_dict=feed_dict)
            #all_tensors = tf.contrib.graph_editor.get_tensors(tf.get_default_graph())
            #all_tensor_names = [tensor.name for tensor in all_tensors]
            #print (all_tensor_names)
            feed_dict = {images_placeholder: images}
            facenet_emb_list = sess.run(output_tensor, feed_dict=feed_dict)

Model directory: /home/ovuser/FaceRecognition/models/20170512-110547
Metagraph file: model-20170512-110547.meta
Checkpoint file: model-20170512-110547.ckpt-250000
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from /home/ovuser/FaceRecognition/models/20170512-110547/model-20170512-110547.ckpt-250000
Tensor("Shape_5:0", shape=(4,), dtype=int32)


ValueError: Shapes must be equal rank, but are 1 and 4 for 'Assign_1' (op: 'Assign') with input shapes: [32], [3,3,3,32].

In [6]:
print (emb_list)

[[[[75.32163     0.          0.         ...  0.         23.59276
     0.        ]
   [74.29479     0.          0.         ...  0.         22.440212
     0.        ]
   [74.89318     0.          0.         ...  0.         21.934326
     0.        ]
   ...
   [71.14871     0.          0.         ...  0.         23.772858
     0.        ]
   [72.07695     0.          0.         ...  0.         23.078848
     0.        ]
   [72.77179     0.          0.         ...  0.         23.10966
     0.        ]]

  [[74.065506    0.          0.         ...  0.         24.118254
     0.        ]
   [73.17446     0.          0.         ...  0.         23.218664
     0.        ]
   [74.82022     0.          0.         ...  0.         21.92794
     0.        ]
   ...
   [71.4891      0.          0.         ...  0.         24.001389
     0.        ]
   [71.51665     0.          0.         ...  0.         23.601269
     0.        ]
   [71.89565     0.          0.         ...  0.         22.943272
     0. 